In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bayes_opt import BayesianOptimization
sns.set()

In [2]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    res = []
    for i in range(n - 1):
        res.append(block[i + 1] - block[i])
    return np.array([res])

In [3]:
df = pd.read_csv('GNUS.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-02-11,2.280,2.290,2.22,2.22,2.22,9000
1,2019-02-12,2.296,2.296,2.20,2.23,2.23,5200
2,2019-02-13,2.260,2.320,2.12,2.32,2.32,52600
3,2019-02-14,2.203,2.250,2.19,2.20,2.20,38800
4,2019-02-15,2.150,2.150,1.63,2.03,2.03,185700


In [4]:
close = df.Close.values.tolist()
window_size = 30
skip = 5
l = len(close) - 1
close

[2.22,
 2.23,
 2.32,
 2.2,
 2.03,
 2.08,
 2.06,
 2.05,
 2.1,
 2.09,
 2.04,
 2.04,
 2.09,
 2.05,
 2.1,
 2.06,
 2.06,
 1.99,
 2.0]

In [5]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (
                    w
                    + self.learning_rate
                    / (self.population_size * self.sigma)
                    * np.dot(A.T, rewards).T
                )
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, self.reward_function(self.weights))
                )
        print('time taken to train:', time.time() - lasttime, 'seconds')


class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size),
            np.random.randn(layer_size, output_size),
            np.random.randn(layer_size, 1),
            np.random.randn(1, layer_size),
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        buy = np.dot(feed, self.weights[2])
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [6]:
class Agent:
    def __init__(
        self,
        population_size,
        sigma,
        learning_rate,
        model,
        money,
        max_buy,
        max_sell,
        skip,
        window_size,
    ):
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
        )

    def act(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = get_state(close, 0, self.window_size + 1)
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell

            state = next_state
        return ((initial_money - starting_money) / starting_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    def buy(self):
        initial_money = self.initial_money
        state = get_state(close, 0, self.window_size + 1)
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        for t in range(0, l, self.skip):
            action, buy = self.act(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                print(
                    'day %d: buy %d units at price %f, total balance %f'
                    % (t, buy_units, total_buy, initial_money)
                )
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                if sell_units < 1:
                    continue
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                states_sell.append(t)
                try:
                    invest = ((total_sell - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (t, sell_units, total_sell, invest, initial_money)
                )
            state = next_state

        invest = ((initial_money - starting_money) / starting_money) * 100
        print(
            '\ntotal gained %f, total investment %f %%'
            % (initial_money - starting_money, invest)
        )
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'true close', c = 'g')
        plt.plot(
            close, 'X', label = 'predict buy', markevery = states_buy, c = 'b'
        )
        plt.plot(
            close, 'o', label = 'predict sell', markevery = states_sell, c = 'r'
        )
        plt.legend()
        plt.show()

In [7]:
def best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    model = Model(window_size, size_network, 3)
    agent = Agent(
        population_size,
        sigma,
        learning_rate,
        model,
        10000,
        5,
        5,
        skip,
        window_size,
    )
    try:
        agent.fit(10, 1000)
        return agent.es.reward_function(agent.es.weights)
    except:
        return 0

In [8]:
def find_best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    global accbest
    param = {
        'window_size': int(np.around(window_size)),
        'skip': int(np.around(skip)),
        'population_size': int(np.around(population_size)),
        'sigma': max(min(sigma, 1), 0.0001),
        'learning_rate': max(min(learning_rate, 0.5), 0.000001),
        'size_network': int(np.around(size_network)),
    }
    print('\nSearch parameters %s' % (param))
    investment = best_agent(**param)
    print('stop after 100 iteration with investment %f' % (investment))
    if investment > accbest:
        costbest = investment
    return investment

In [9]:
accbest = 0.0
NN_BAYESIAN = BayesianOptimization(
    find_best_agent,
    {
        'window_size': (2, 50),
        'skip': (1, 15),
        'population_size': (1, 50),
        'sigma': (0.01, 0.99),
        'learning_rate': (0.000001, 0.49),
        'size_network': (10, 1000),
    },
)
NN_BAYESIAN.maximize(init_points = 30, n_iter = 50, acq = 'ei', xi = 0.0)

|   iter    |  target   | learni... | popula... |   sigma   | size_n... |   skip    | window... |
-------------------------------------------------------------------------------------------------

Search parameters {'window_size': 34, 'skip': 3, 'population_size': 20, 'sigma': 0.6770146704505254, 'learning_rate': 0.3977786862040309, 'size_network': 269}


C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


stop after 100 iteration with investment 0.000000
|  1        |  0.0      |  0.3978   |  19.52    |  0.677    |  268.8    |  3.483    |  34.11    |

Search parameters {'window_size': 10, 'skip': 8, 'population_size': 47, 'sigma': 0.048156567757646575, 'learning_rate': 0.20971744848057539, 'size_network': 737}
stop after 100 iteration with investment 0.000000
|  2        |  0.0      |  0.2097   |  46.69    |  0.04816  |  736.9    |  7.606    |  10.05    |

Search parameters {'window_size': 25, 'skip': 5, 'population_size': 8, 'sigma': 0.44070501970936443, 'learning_rate': 0.4706676215039481, 'size_network': 415}
stop after 100 iteration with investment 0.000000
|  3        |  0.0      |  0.4707   |  7.627    |  0.4407   |  414.9    |  5.04     |  25.28    |

Search parameters {'window_size': 29, 'skip': 11, 'population_size': 13, 'sigma': 0.569966080498838, 'learning_rate': 0.21310072734943442, 'size_network': 933}
stop after 100 iteration with investment 0.000000
|  4        |  0.0    

time taken to train: 0.7583343982696533 seconds
stop after 100 iteration with investment 0.000000
|  23       |  0.0      |  0.405    |  35.53    |  0.975    |  447.8    |  8.483    |  25.74    |

Search parameters {'window_size': 22, 'skip': 14, 'population_size': 1, 'sigma': 0.013296271900300994, 'learning_rate': 0.28379903812729557, 'size_network': 585}
stop after 100 iteration with investment 0.000000
|  24       |  0.0      |  0.2838   |  1.466    |  0.0133   |  584.6    |  14.12    |  22.24    |

Search parameters {'window_size': 6, 'skip': 11, 'population_size': 32, 'sigma': 0.7103186759248623, 'learning_rate': 0.429026065330498, 'size_network': 449}
stop after 100 iteration with investment 0.000000
|  25       |  0.0      |  0.429    |  31.96    |  0.7103   |  449.0    |  10.71    |  5.706    |

Search parameters {'window_size': 19, 'skip': 14, 'population_size': 21, 'sigma': 0.3371780879752772, 'learning_rate': 0.19590605338071174, 'size_network': 456}
stop after 100 iteration

C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


...

C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide



Search parameters {'window_size': 2, 'skip': 1, 'population_size': 4, 'sigma': 0.46171298495463864, 'learning_rate': 0.315888124889866, 'size_network': 31}
stop after 100 iteration with investment 0.000000
|  79       |  0.0      |  0.3159   |  4.401    |  0.4617   |  31.19    |  1.217    |  2.392    |


C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide



Search parameters {'window_size': 49, 'skip': 15, 'population_size': 47, 'sigma': 0.4729894682805449, 'learning_rate': 0.2592588915902506, 'size_network': 25}
stop after 100 iteration with investment 0.000000
|  80       |  0.0      |  0.2593   |  47.06    |  0.473    |  24.54    |  14.75    |  49.47    |


C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


In [11]:
print('Best AGENT accuracy value: %f' % NN_BAYESIAN.res['max']['max_val'])
print('Best AGENT parameters: ', NN_BAYESIAN.res['max']['max_params'])

TypeError: list indices must be integers or slices, not str

In [14]:
# this is my parameter
best_agent(30, 1, 15, 0.1, 0.03, 500)

time taken to train: 8.902231216430664 seconds


44.664403540000016

In [32]:
# this is from bayesian
best_agent(20, 1, 34, 0.7152490670931403, 0.28208006704411104, 176)

[[ -8.08350637 -34.28026983  -2.66784998]] [[-8.85968201]]
[[-21.42116526 -43.36871289 -12.98324049]] [[-5.27531589]]
[[-31.20257693 -45.73055655 -68.41548793]] [[-21.51502866]]
[[ -26.54570657    1.33010818 -168.56409898]] [[-102.46715733]]
[[ -31.06672918   82.78805877 -118.08289735]] [[-196.65620346]]
[[-360.04687086 -187.98393899  135.59540435]] [[55.14417243]]
[[ 216.06513331 -122.13677575  119.63899647]] [[150.48276225]]
[[   7.14853009  -92.13262333 -406.71426577]] [[74.40116283]]
[[  66.77509856  229.2818813  -358.59636441]] [[-369.44265858]]
[[-600.97350028 -108.32056746   58.56259128]] [[-123.93845594]]
[[-170.00830939 -209.24221528  432.56182275]] [[375.15145919]]
[[ 694.40112285 -344.58423664   84.58451989]] [[224.19035294]]
[[-523.94736014 -218.45031875  206.10602357]] [[566.83308402]]
[[ 376.95589231 -356.21165995  158.36921279]] [[71.32343046]]
[[  637.63181439   984.4030414  -1272.6804366 ]] [[-495.40771381]]
[[-1627.69514607   -67.40342874  -908.40170067]] [[-1213.6005

[[ 5.2403886   5.84244791 18.69444227]] [[-12.94196862]]
[[-43.88864637  11.08161132   8.41770921]] [[-19.5019908]]
[[-153.77231734   19.44659911  -51.2847492 ]] [[-61.83650613]]
[[-189.05967619   15.2976115  -171.61640499]] [[-152.09001023]]
[[ 139.7857799    -1.53611605 -200.15471167]] [[-180.07637911]]
[[-266.61113612  139.66128576  -86.43846877]] [[-67.84941589]]
[[ 175.42710317  146.42885246 -170.95010662]] [[-107.25926068]]
[[-402.82973084  101.64185246 -318.649675  ]] [[-234.6531114]]
[[ 189.82677329   -3.02368898 -323.13187942]] [[-327.42442417]]
[[ -52.23570879  142.39910086 -289.61685979]] [[-146.95895063]]
[[101.54534008 233.42497943 -58.40160701]] [[252.1044666]]
[[1256.96710295 -203.71170777   -4.20125647]] [[-263.10716954]]
[[-87.69477182 124.71115286 857.17261303]] [[480.51843831]]
[[-422.47460944  255.56758726  271.71003175]] [[101.74435171]]
[[  94.12592925 -320.03976167 -489.21201313]] [[-52.61635047]]
[[-2233.0125435   -865.12685381  -400.23954672]] [[-640.29116358]]

[[-1.98894647 30.42135038 -0.72367398]] [[-22.70645087]]
[[-25.93093438  77.53756869  -7.34410359]] [[-66.23781306]]
[[-14.70273348 102.41049577   6.07309312]] [[-80.07725325]]
[[ 61.21763011 -11.21493851  40.04428829]] [[43.07826315]]
[[-276.4385812   163.82039698 -149.69967218]] [[-171.59824432]]
[[228.84328022 126.49901641   2.97567539]] [[-41.94098559]]
[[-153.02089574  240.42730646 -218.12501664]] [[-227.8853882]]
[[ -42.58054583 -116.31092662   -0.77720472]] [[266.11822889]]
[[-214.47548091  219.078466    -60.49324931]] [[9.30352913]]
[[   2.35985106  161.37231122 -204.36101859]] [[194.1458411]]
[[ 920.90590726 -576.51597333  111.61673961]] [[219.53689891]]
[[-1134.22870196  -361.20943526  -630.00644036]] [[277.04314992]]
[[157.5889717  515.40539567 253.68499881]] [[-442.4971078]]
[[712.36812623 410.92502849 406.4123957 ]] [[588.36273692]]
[[-1193.99076678  -458.07427989  -126.85687647]] [[46.29349811]]
[[-237.8246461   -93.03158481  605.10492727]] [[-297.41570993]]
[[ 688.026391

[[  3.34777963 -73.86024992 -19.67692444]] [[-70.91343579]]
[[-236.63207733  119.63945203   73.83108237]] [[29.89703884]]
[[ 66.81637303 337.4565123   73.99054436]] [[-206.39107413]]
[[ -91.06769415  391.36737084 -370.32451602]] [[-189.38040583]]
[[ 243.72132102   11.8799083  -412.68240882]] [[-219.11296807]]
[[-140.86358449  -63.20799519  253.83239501]] [[366.50624088]]
[[-138.40728567  293.58332177  177.02535412]] [[-83.01179768]]
[[ 247.88889059 -247.23576015   83.44605378]] [[-357.52825607]]
[[-232.10817513 -113.39320038 -276.84870795]] [[32.81809712]]
[[ 731.95849005  674.80649294 -146.74630409]] [[159.1665999]]
[[ 487.60745407 -178.51668268 -741.25959764]] [[196.15668181]]
[[ -806.13743049 -1211.618945   -1495.22334808]] [[83.35011965]]
[[-309.18230545   -8.21736107  473.65748727]] [[-65.90703863]]
[[-627.24932834  948.39235206  847.81226754]] [[-782.19548623]]
[[ 4.47874886 -5.3139823  -2.72865244]] [[-4.26561059]]
[[-17.10691657 -14.41054886  -5.69600797]] [[-4.2350413]]
[[-62.

[[ 89.57821277 141.9836     -96.48631998]] [[-90.36769612]]
[[ 193.90913175  244.8552213  -198.37141169]] [[-158.62077235]]
[[  51.11393151   22.09188844 -141.02172586]] [[-80.2760503]]
[[-372.87984239   81.38503437  -36.74715845]] [[-83.37532788]]
[[150.90391874 199.17559005  45.74824604]] [[-41.327573]]
[[ 396.53311407  391.81675111 -368.00299929]] [[-90.55987195]]
[[ 398.98635936  152.55410596 -412.26074641]] [[-224.35242707]]
[[-479.26318691  -55.88498326  -90.8094173 ]] [[135.0734032]]
[[-452.23723846  -87.87957374  271.93773072]] [[92.00866054]]
[[ 109.52387299 -744.24075083  320.16469319]] [[551.46492693]]
[[-751.65915404 -879.86065635  240.65076665]] [[244.01561791]]
[[695.69797063 702.7593048   21.06384023]] [[-142.73705238]]
[[1979.0675106   758.64883189 -540.1073052 ]] [[100.81414536]]
[[ -283.48044731    24.54360306 -1439.81109815]] [[-778.24374346]]
[[-653.63911182  876.05591442  -74.06840883]] [[-1180.32813075]]
[[-298.29694417  338.78820699   41.33837684]] [[-181.9042377

...

[[  2.35405702  70.83437849 535.94680041]] [[317.62199254]]
[[ 273.13784752 -298.36371679 -651.93462871]] [[792.70332711]]
[[-467.5134315  -171.71109129  605.21049065]] [[227.89586743]]
[[633.92676064 322.63268717 948.98942004]] [[-149.72249406]]
[[ 1239.1671795  -1031.59131702 -2092.40223617]] [[-190.07727986]]
[[-489.33162809 -735.16376403 -384.95192925]] [[-1647.36985929]]
[[-882.80907059  194.10711442 1056.87141214]] [[-1151.0143787]]
[[ 299.64486334  543.3148774  -119.58734873]] [[72.31735817]]
[[ 27.01768843 -10.93719193 -21.12496041]] [[4.82836527]]
[[41.42604937 -6.45318656 15.42717257]] [[-12.11235476]]
[[79.00551414  3.57662514 86.17619654]] [[-75.49370146]]
[[ 56.6438949  -29.85436807  32.7507199 ]] [[-167.49483029]]
[[-164.58495832 -153.09483944 -316.64054427]] [[-115.35218376]]
[[-117.34188246 -117.34409674  145.63383257]] [[-77.9592903]]
[[271.29531904 220.06380644 245.60726521]] [[43.72093082]]
[[350.34948773  98.87445627  67.95708089]] [[-161.39169741]]
[[-132.83591024 

[[54.73233203  2.43934382  2.32006644]] [[-14.27294422]]
[[ 69.50910923  26.89687724 -54.35628121]] [[-3.04568996]]
[[-67.88805253  33.97631072 -73.39267516]] [[45.79096045]]
[[-40.03648854 -60.54292774 243.56784928]] [[-47.5886196]]
[[ 517.75117245  158.23870265 -178.98218974]] [[86.46564197]]
[[405.42786067 132.26790859 -67.50755586]] [[-90.40536577]]
[[-157.71069864   43.57260981 -384.2648176 ]] [[-34.88379359]]
[[  90.40949162 -131.26639805  384.99338871]] [[65.27048967]]
[[267.4677477   35.41162413 -55.24860082]] [[217.83869102]]
[[ 710.34263746  261.0061503  -298.76007635]] [[232.25565466]]
[[-547.5962135  -299.47807105  253.49444214]] [[-449.56560631]]
[[ 906.64885567  -61.64034124 -182.60106544]] [[-363.09619001]]
[[  869.59567034   232.0878259  -1050.83880923]] [[408.627139]]
[[-2333.23356071  -748.18957537   666.74520185]] [[-215.12551401]]
[[ -76.56654815 -232.26995845  266.51478958]] [[363.83429973]]
[[1412.18250711  531.24263525 -450.1781942 ]] [[292.78633527]]
[[ -1.18403

[[  833.31380048  -220.82261789 -1068.11541825]] [[723.05496287]]
[[-1255.49580486 -2477.06732791   234.94680061]] [[-660.00092119]]
[[ -830.23492832 -1434.67603418   972.42093873]] [[-1169.26900688]]
[[454.21128279 -50.11063955 486.22961782]] [[-1080.05092866]]
[[ 14.44426847 -14.0496397  -19.85538656]] [[4.27839058]]
[[ 11.7830455  -21.62697557 -30.43988553]] [[-24.8870551]]
[[ 25.26213284 -60.66313887 -82.9133303 ]] [[-97.71043999]]
[[  38.97279541 -134.95021669 -149.75805076]] [[-140.89753107]]
[[ -46.57244459 -116.77549651  -44.93921642]] [[38.86433704]]
[[-313.8688289    41.16910973  144.16332789]] [[-191.81660561]]
[[117.42507415  99.81316344 -91.54201947]] [[-67.15519394]]
[[ 150.71368992 -237.24138537 -363.26743587]] [[-478.35778532]]
[[ -23.12834515 -294.0772926  -215.46530331]] [[91.88761954]]
[[-227.49633502  -15.64506968   76.00045375]] [[-249.40641356]]
[[-298.95572364  273.50425968  346.84388026]] [[39.55760203]]
[[ 637.3801959    89.47727282 -235.74466957]] [[250.374614

[[-294.42143529 -300.19298458 1048.62333796]] [[-1209.62385333]]
[[654.97127099  66.57219937 -26.08346111]] [[-1384.90248571]]
time taken to train: 6.122346639633179 seconds
[[ 12.92661144 -10.76991648  -0.66765682]] [[-14.46368465]]
[[ 13.65964828 -15.86315789  -2.28480891]] [[-23.0427918]]
[[ 31.40817511 -20.48176399 -28.75858942]] [[-60.77448367]]
[[ 45.80673563 -16.57177965 -85.50678312]] [[-130.5177019]]
[[-28.44832654  -3.44604335 -71.46347546]] [[-132.27391795]]
[[-171.66395401 -110.61798632  126.95077   ]] [[-88.16374858]]
[[216.90386094  58.41861219 -60.9262617 ]] [[-29.90394414]]
[[ 196.3670552   -49.9311597  -249.41730191]] [[-231.32732614]]
[[ -61.50794767   19.97325018 -201.43638062]] [[-177.52700583]]
[[-198.19725924 -160.3591108   207.01263853]] [[-195.4295429]]
[[-72.88609386 -66.22567155 186.44843625]] [[171.92065568]]
[[ 554.07928469  -22.78719257 -264.26693972]] [[11.71322299]]
[[-547.5396838  -242.40678375  227.26008765]] [[223.18148587]]
[[558.97155755 143.22230094

0.018200080000024172

In [ ]:
# this is my parameter
model = Model(30, 500, 3)
agent = Agent(15, 0.1, 0.03, model, 10000, 5, 5, 1, 30)
agent.fit(500, 100)
agent.buy()

In [ ]:
# this is from bayesian
model = Model(20, 176, 3)
agent = Agent(
    34, 0.7152490670931403, 0.28208006704411104, model, 10000, 5, 5, 1, 20
)
agent.fit(500, 100)
agent.buy()